In [1]:
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
import matplotlib as mpl
import sklearn
from sklearn import metrics
from tensorflow.keras.utils import plot_model
from tensorflow.keras import backend as K
import random


def dataModelPreparation():
    # 本地读取MNIST数据
    path = './dataset/mnist.npz'
    f = np.load(path)
    x_train, y_train = f['x_train'], f['y_train']
    x_test, y_test = f['x_test'], f['y_test']
    f.close()

    # 为便于评测，图像尺寸缩小为原来的一半
    h = x_train.shape[1] // 2
    w = x_train.shape[2] // 2

    x_train = np.expand_dims(x_train, axis=-1)
    x_train = tf.image.resize(x_train, [h, w]).numpy()  # if we want to resize
    x_test = np.expand_dims(x_test, axis=-1)
    x_test = tf.image.resize(x_test, [h, w]).numpy()  # if we want to resize

    # 图像归一化,易于网络学习
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255.0
    x_test /= 255.0

    # WORK1: --------------BEGIN-------------------
    # 请补充完整训练集和测试集的产生方法：
    n_classes = 10
    # 请将训练集和测试集的标签转换为one-hot编码
    y_train = tf.keras.utils.to_categorical(y_train, n_classes)
    y_test = tf.keras.utils.to_categorical(y_test, n_classes)

    # 请补充建立训练集和测试集的代码
    train_datasets = tf.data.Dataset.zip(
        (
            tf.data.Dataset.from_tensor_slices(x_train).batch(60000),
            tf.data.Dataset.from_tensor_slices(y_train).batch(60000)
        )
    )

    test_datasets = tf.data.Dataset.zip(
        (
            tf.data.Dataset.from_tensor_slices(x_test).batch(10000),
            tf.data.Dataset.from_tensor_slices(y_test).batch(10000)
        )
    )
    # WORK1: ---------------END--------------------

    # WORK2: --------------BEGIN-------------------
    # 请参考所给网络结构图，补充完整共享参数孪生网络siamese_net的实现：
    # 注意，我们用比较图片的方法来评测网络结构是否正确
    # 所以网络结构中的参数维度、名称等需和参考图中一致，否则不能通过评测
    inputs = tf.keras.Input(shape=(14, 14, 1), name='data')
    conv1 = tf.keras.layers.Conv2D(filters=16, kernel_size=3, strides=1, padding='valid', activation='relu',name='conv1')(inputs)
    pool1 = tf.keras.layers.MaxPool2D(pool_size=2, strides=2, name='pool1')(conv1)
    conv2 = tf.keras.layers.Conv2D(filters=16, kernel_size=3, strides=1, padding='same', activation='relu',name='conv2')(pool1)
    pool2 = tf.keras.layers.MaxPool2D(pool_size=2, strides=2, name='pool2')(conv2)
    flat = tf.keras.layers.Flatten(name='flat')(pool2)
    fc1 = tf.keras.layers.Dense(84, activation='relu',name='fc1')(flat)
    outputs = tf.keras.layers.Dense(10, activation='softmax', name='output')(fc1)


    # WORK2: ---------------END--------------------

    model = tf.keras.Model(inputs=inputs, outputs=outputs, name='conv_net')

    plot_model(model, to_file='./test_figure/step1/conv_net.png', show_shapes=True, expand_nested=True)

    return train_datasets, test_datasets, model


# WORK3: --------------BEGIN-------------------
# 实例化网络并进行训练
def test_fun():
    train_datasets, test_datasets, model = dataModelPreparation()
    # 3.1 model compile，请选择适用于图像分类任务及one-hot编码的loss
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
        loss=tf.nn.sigmoid_cross_entropy_with_logits,
        metrics=['acc']
    )
    # 3.2 配置训练参数，开始训练，
    history = model.fit(
        train_datasets,
        validation_data=test_datasets,
        batch_size=64,
        epochs=40,
        verbose=2
    )
    return model, history
# WORK3: ---------------END--------------------
test_fun()
########### 以下为测试代码，评测时自动调用，请不要取消注释！！！ #####
# from siamese_work import test_fun
# import matplotlib.pyplot as plt
# import matplotlib.image as mpimg
# import numpy as np

# import os

# model, history = test_fun()

# 1.用图片对比的方法测试网络结构是否正确
# test_img = mpimg.imread('./test_figure/step1/conv_net.png')
# answer_img= mpimg.imread('./path/to/answer.png')
# assert((answer_img == test_img).all())
# print('Network pass!')


# 2.测试网络训练是否达标
# if history.history['val_acc'][-1] > 0.90:
#     print("Success!")

Epoch 1/40
1/1 - 2s - loss: 0.7345 - acc: 0.0550 - val_loss: 0.7339 - val_acc: 0.1279 - 2s/epoch - 2s/step
Epoch 2/40
1/1 - 1s - loss: 0.7339 - acc: 0.1326 - val_loss: 0.7329 - val_acc: 0.2960 - 1s/epoch - 1s/step
Epoch 3/40
1/1 - 1s - loss: 0.7329 - acc: 0.2953 - val_loss: 0.7310 - val_acc: 0.2032 - 888ms/epoch - 888ms/step
Epoch 4/40
1/1 - 1s - loss: 0.7310 - acc: 0.2035 - val_loss: 0.7281 - val_acc: 0.2286 - 1s/epoch - 1s/step
Epoch 5/40
1/1 - 1s - loss: 0.7281 - acc: 0.2260 - val_loss: 0.7252 - val_acc: 0.2654 - 937ms/epoch - 937ms/step
Epoch 6/40
1/1 - 1s - loss: 0.7254 - acc: 0.2656 - val_loss: 0.7206 - val_acc: 0.3218 - 999ms/epoch - 999ms/step
Epoch 7/40
1/1 - 1s - loss: 0.7208 - acc: 0.3195 - val_loss: 0.7178 - val_acc: 0.3754 - 946ms/epoch - 946ms/step
Epoch 8/40
1/1 - 1s - loss: 0.7181 - acc: 0.3692 - val_loss: 0.7122 - val_acc: 0.5488 - 866ms/epoch - 866ms/step
Epoch 9/40
1/1 - 1s - loss: 0.7126 - acc: 0.5418 - val_loss: 0.7088 - val_acc: 0.5316 - 820ms/epoch - 820ms/step
E

(<keras.engine.functional.Functional at 0x1d7c8629d00>,
 <keras.callbacks.History at 0x1d7c86b7f40>)

In [29]:
#WORK3: --------------BEGIN-------------------
#实例化网络并进行训练
def test_fun():
    train_datasets, test_datasets, model = dataModelPreparation()
    #3.1 model compile，请选择适用于图像分类任务及one-hot编码的loss
    model.compile(
        optimizer=tf.optimizers.Adam(0.01),
        loss=tf.losses.categorical_crossentropy,
        metrics=['acc']
    )
    #3.2 配置训练参数，开始训练，
    history = model.fit(
    train_datasets,
    batch_size = 128,
    validation_data = test_datasets,
    epochs = 15,
    verbose = 2
    )
    return model, history
#WORK3: ---------------END--------------------

(10000, 14, 14, 1)
Epoch 1/15
1/1 - 2s - loss: 2.3083 - acc: 0.1237 - val_loss: 2.2873 - val_acc: 0.0985 - 2s/epoch - 2s/step
Epoch 2/15
1/1 - 1s - loss: 2.2884 - acc: 0.0992 - val_loss: 2.0812 - val_acc: 0.4719 - 752ms/epoch - 752ms/step
Epoch 3/15
1/1 - 1s - loss: 2.0893 - acc: 0.4563 - val_loss: 1.9548 - val_acc: 0.4764 - 741ms/epoch - 741ms/step
Epoch 4/15
1/1 - 1s - loss: 1.9669 - acc: 0.4573 - val_loss: 1.7500 - val_acc: 0.5426 - 817ms/epoch - 817ms/step
Epoch 5/15
1/1 - 1s - loss: 1.7664 - acc: 0.5298 - val_loss: 1.4757 - val_acc: 0.7030 - 784ms/epoch - 784ms/step
Epoch 6/15
1/1 - 1s - loss: 1.4964 - acc: 0.6798 - val_loss: 1.1858 - val_acc: 0.7715 - 887ms/epoch - 887ms/step
Epoch 7/15
1/1 - 1s - loss: 1.2106 - acc: 0.7588 - val_loss: 0.9354 - val_acc: 0.7878 - 843ms/epoch - 843ms/step
Epoch 8/15
1/1 - 1s - loss: 0.9618 - acc: 0.7757 - val_loss: 0.7429 - val_acc: 0.8073 - 827ms/epoch - 827ms/step
Epoch 9/15
1/1 - 1s - loss: 0.7661 - acc: 0.7979 - val_loss: 0.6311 - val_acc: 0.81